# And how many people live in your country?

Input (or paste) name of the countries in the left textbox, clik the button and population for each of the countries will (well, shall) be displayed in the right textbox.

Example of valid input:

Belgium
Slovenia
Zimbabve

<small>Population is retrieved using "Determine total population for a given country on a given date" service found here: http://api.population.io/#!/population/determineTotalPopulationByDate</small>

In [42]:
import ipywidgets as ipw
import requests

countries = ipw.Textarea(placeholder="Give me some countries", layout=ipw.Layout(width="150px", height="250px"))
people = ipw.Textarea(placeholder="", layout=ipw.Layout(width="150px", height="250px"))

def on_click(btn):
    people.value = ''
    countries_split = countries.value.split('\n')
    for country in countries_split:
        if country:
            try:
                people.value += (get_country_population(country) + '\n')
            except:
                people.value += (' At least one of the values could not be obtained. Check if you have correct country names.' + '\n')
                return
        else:
            people.value += ('' + '\n')
        
def get_country_population(country):
    try:
        r = requests.get('http://api.population.io:80/1.0/population/'+str(country)+'/2018-12-12/') #watchout: hardcoded date.
        population = str(r.json()['total_population']['population'])
        r.raise_for_status()
        if r.status_code != 200:
            print(r.status_code)
        return population
    except:
        people.value = "ERROR"

def mk_btn(description):
    btn = ipw.Button(description=description, layout=ipw.Layout(width="300px"))
    btn.on_click(on_click)
    return btn

row0 = ipw.HBox([countries, people])
row1 = ipw.HBox([mk_btn("Get popultation")])

ipw.VBox((row0, row1))

A Jupyter Widget